### Instruction:
Summarize following text.

### Input:
Text to be summarized

### Response:

In [3]:
#!pip install sentencepiece
#!pip install tokenizers -U
#!pip install transformers[sentencepiece] -U
#!pip install accelerate -U
#!pip install optimum

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1" #auto gets confused for some reason and puts it on the 2080
import torch
torch.cuda.device_count()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.device_count()
torch.cuda.get_device_name(device=None)

'NVIDIA GeForce RTX 3090 Ti'

In [2]:
#%pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM 
import transformers
import torch
import sentencepiece

model = "NousResearch/Nous-Hermes-13b"

#https://huggingface.co/docs/transformers/perf_infer_gpu_one
#https://pytorch.org/blog/out-of-the-box-acceleration/
#model = model.to_bettertransformer()
#takes forever to load?
tokenizer = AutoTokenizer.from_pretrained(model)
#https://github.com/huggingface/transformers/blob/main/src/transformers/pipelines/text_generation.py
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    #device_map= 1, #auto gets confused for some reason and puts it on the 2080
    device="cuda:0",
    max_new_tokens=100,
    return_full_text=False,
    batch_size=1
)

/home/skynet3/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OutOfMemoryError: CUDA out of memory. Tried to allocate 136.00 MiB (GPU 0; 23.68 GiB total capacity; 22.81 GiB already allocated; 79.12 MiB free; 22.81 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
print(pipeline(["Answer the following question:\nQ.What is the capital for Italy?\nA."]))


In [5]:
prompt="""### Instruction:
Summarize following text.

### Input:
The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]

### Response:"""

In [6]:
result=pipeline([prompt])
print(result)

[[{'generated_text': '### Instruction:\nSummarize following text.\n\n### Input:\nThe Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]\n\n### Response:The Cuban Missile Crisis was a 13-day confrontation between the US and the USSR in 1962, during which both sides

In [17]:
prompt="""### Instruction:
Create bulleted list the all of the countries and organizations mentioned in this sentence.

### Input:
The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.

### Response:"""


In [18]:
result=pipeline([prompt])
print(result)

[[{'generated_text': '### Instruction:\nCreate bulleted list the all of the countries and organizations mentioned in this sentence.\n\n### Input:\nThe Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.\n\n### Response:Countries:\n- Russia\n- Germany\n\nOrganizations:\n- Communist regime\n- World War I\n- Treaty of Brest-Litovsk'}]]


In [21]:
prompt="""### Instruction:
Create semicolon separated list the all of the countries and organizations mentioned in this sentence.

### Input:
The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok.

### Response:
Western Allies; Germany

### Input:
The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.

### Response:"""

In [22]:
result=pipeline([prompt])
print(result)

/home/skynet3/anaconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[[{'generated_text': '### Instruction:\nCreate semicolon separated list the all of the countries and organizations mentioned in this sentence.\n\n### Input:\nThe Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok.\n\n### Response:\nWestern Allies; Germany\n\n### Input:\nThe Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.\n\n### Response:\nCommunist regime; Russia; Germany'}]]


In [30]:
prompt="""### Instruction:
Create a semicolon separated list of all of the countries and organizations mentioned in final sentence of the paragraph. Do not include any not mentioned in the final sentence. Do not include entities mentioned earlier but not in the final sentence.

### Input:
The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.

### Response:
Communist regime; Russia; Germany

### Input:
The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918. The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok.

### Response:
Western Allies; Germany

### Input:
The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918. The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok. In pursuit of these goals the Allies attempted to use the 40,000 troops of the Czech. Legion then in Siberia.

### Response:
"""


In [31]:
result=pipeline([prompt])
print(result)

/home/skynet3/anaconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[[{'generated_text': '### Instruction:\nCreate a semicolon separated list of all of the countries and organizations mentioned in final sentence of the paragraph. Do not include any not mentioned in the final sentence. Do not include entities mentioned earlier but not in the final sentence.\n\n### Input:\nThe Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.\n\n### Response:\nCommunist regime; Russia; Germany\n\n### Input:\nThe Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918. The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok.\n\n### Response:\nWestern Allies; Germany\n\n### Input:\nThe Communist reg

In [5]:
prompt="""### Instruction:
Create a semicolon separated list of all of the countries and organizations mentioned in final sentence of the paragraph. Explain your reasoning and then give a final answer.

### Input:
The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.

### Response:
Thought: The paragraph contains only one sentence, and it refers to organization "The Communist regime" and the country Germany.

Final Answer: Communist regime; Russia; Germany

### Input:
The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918. The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok.

### Response:
Thought: The final sentence is "The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok." which mentions the organization Western Allies and the country Germany.

Final Answer: Western Allies; Germany

### Input:
The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918. The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok. In pursuit of these goals the Allies attempted to use the 40,000 troops of the Czech. Legion then in Siberia.

### Response:
"""

In [ ]:
result=pipeline([prompt])
print(result)

In [3]:
prompt="""### Instruction:
Create a semicolon separated list of all of the countries and organizations mentioned in final sentence of the paragraph. Explain your reasoning and then give a final answer.

### Input:
The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918. The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok.

### Response:
Thought: The final sentence is "The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok." That final sentence contains the named entities Western Allies, Germany, and Vladivostok. Of those, Western Allies and Germany are countries.

Final Answer: Western Allies; Germany 

### Input:
The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918. The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok. In pursuit of these goals the Allies attempted to use the 40,000 troops of the Czech. Legion then in Siberia.

### Response:
"""
print(len(prompt)/4)

361.5


In [4]:
result=pipeline([prompt])
print(result)

OutOfMemoryError: CUDA out of memory. Tried to allocate 22.00 MiB (GPU 0; 23.68 GiB total capacity; 22.44 GiB already allocated; 13.12 MiB free; 22.49 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF